In [3]:
import requests
from bs4 import BeautifulSoup
import json
import time 
import re
import pandas as pd
import mysql.connector
import pymysql

In [8]:
url ='https://www.kurashiru.com/recipes/459512b1-1c95-4a59-917d-4363f440d0a7'
def get_html(url):
    time.sleep(2)
    res = requests.get(url)
    res.encoding = 'utf8' 
    soup = BeautifulSoup(res.text,'html.parser')
    return soup

def get_page(url):
    result = {}
    soup = get_html(url)
    
    json_elem = soup.find(id="hydration_initial_state")
    json_str = get_text(json_elem)
    json_load = json.loads(json_str)
    data_dic = json_load['state']['fetchVideo']['data']['data']
    
    recipe_id = data_dic['id']
    recipe_name = data_dic["attributes"]["title"]
    ingredients = data_dic["attributes"]["ingredients"]
    result["recipe_id"] = recipe_id
    result["recipe_name"] = recipe_name
    
    ingredient_list = []
    for ingredient in ingredients:
        ingredient_dic = {}
        
        ingredient_id = ingredient["id"]
        ingredient_type = ingredient["type"]
                     
        if ingredient_type=="ingredients":
            ingredient_group_id = ingredient["group-id"]
            ingredient_title = ingredient["name"]
            ingredient_name = ingredient["actual-name"]
            ingredient_amount = ingredient["quantity-amount"]
                         
        else:
            ingredient_title = ingredient["title"]
            ingredient_group_id = 0
            ingredient_name = ""
            ingredient_amount = ""
        
        ingredient_dic["ingredient_name"] = ingredient_name
        ingredient_dic["ingredient_amount"] = ingredient_amount
                     
        ingredient_list.append(ingredient_dic)
        result["ingredient_list"] = ingredient_list
    return result
    

def get_text(elem):
    text = elem.string
    text = text.strip()  
    return text

result = get_page(url)
print(result)
ingredint_recipe =result['ingredient_list']
recipe = pd.DataFrame(ingredint_recipe)
recipe

{'recipe_id': '459512b1-1c95-4a59-917d-4363f440d0a7', 'recipe_name': '照り焼きチキン', 'ingredient_list': [{'ingredient_name': '鶏もも肉', 'ingredient_amount': '300g'}, {'ingredient_name': '片栗粉', 'ingredient_amount': '大さじ2'}, {'ingredient_name': '料理酒', 'ingredient_amount': '大さじ1'}, {'ingredient_name': 'しょうゆ', 'ingredient_amount': '大さじ1'}, {'ingredient_name': 'みりん', 'ingredient_amount': '大さじ1'}, {'ingredient_name': '砂糖', 'ingredient_amount': '小さじ1'}, {'ingredient_name': 'サラダ油', 'ingredient_amount': '小さじ2'}, {'ingredient_name': 'ベビーリーフ', 'ingredient_amount': '5g'}]}


,ingredient_name,ingredient_amount
0,鶏もも肉,300g
1,片栗粉,大さじ2
2,料理酒,大さじ1
3,しょうゆ,大さじ1
4,みりん,大さじ1
5,砂糖,小さじ1
6,サラダ油,小さじ2
7,ベビーリーフ,5g


In [9]:
recipe.drop(recipe.index[1])

,ingredient_name,ingredient_amount
0,鶏もも肉,300g
2,料理酒,大さじ1
3,しょうゆ,大さじ1
4,みりん,大さじ1
5,砂糖,小さじ1
6,サラダ油,小さじ2
7,ベビーリーフ,5g


In [5]:
recipe_nu=recipe.assign(protein =[39.4, 2.6, 2.7, 0.2, 0.7, 0, 0.1, 1.4, 0.1],
                                            fat =[9, 0.3, 0.2, 0.1, 0.1, 4, 0, 0, 0],
                                            carbs =[0.2, 4.5, 7.6, 1, 6.8, 0, 0.7,1.4, 0.2],
                                            calorie = [224, 25, 34, 5, 31, 35, 10, 14, 1])
recipe_nu

,ingredient_id,ingredient_name,ingredient_amount,protein,fat,carbs,calorie
0,922,鶏もも肉,300g,51.9,57.3,0.0,702
1,278,片栗粉,大さじ2,0.0,0.0,14.7,61
2,1562,料理酒,大さじ1,0.1,0.0,0.7,10
3,2242,しょうゆ,大さじ1,1.4,0.0,1.4,14
4,42,みりん,大さじ1,0.1,0.0,7.8,43
5,206,砂糖,小さじ1,0.0,0.0,3.0,12
6,3517,サラダ油,小さじ2,0.0,0.0,8.0,71
7,1338,ベビーリーフ,5g,0.1,0.0,0.2,1


In [36]:
print(round(recipe_nu['protein'].sum()/2)),
print(round(recipe_nu['fat'].sum()/2)),
print(round(recipe_nu['carbs'].sum()/2)),
print(round(recipe_nu['calorie'].sum()/2))

39.0
18.0
9.0
362.0


In [67]:
cnx = None

try:
    cnx = mysql.connector.connect(
        user='root',  # ユーザー名
        password='na12199611',  # パスワード
        host='localhost',  # ホスト名(IPアドレス）
        database='recipe'  # データベース名
    )

    cursor = cnx.cursor()

    sql = ('''
    INSERT INTO recipe 
        (recipe_id, recipe_name, preference, category, protein, fat, carbs, calorie)
    VALUES 
         (%s, %s, %s, %s, %s, %s, %s, %s)
    ''')

    data = [('140', '昆布ダシで カブの葉のお漬物', '2', '2','1','0', '4', '16')]

    cursor.executemany(sql, data)
    cnx.commit()

    print(f"{cursor.rowcount} records inserted.")

    cursor.close()

except Exception as e:
    print(f"Error Occurred: {e}")

finally:
    if cnx is not None and cnx.is_connected():
        cnx.close()

1 records inserted.


In [15]:
recipe_nu=recipe.assign(protein =[28.8, 0.2, 24.4, 0, 0.7, 0.1, 0, 0, 2.1,0, 0.1, 0.4],
                                            fat =[70.8, 0.1, 20.4, 0, 0.1, 0, 0, 0, 0,0,0,0],
                                            carbs =[0.2, 1, 0.8, 0, 6.8, 0.7, 6, 5.2, 2.1,8, 0.6, 2.9],
                                            calorie = [732, 5, 284, 0, 31, 10,24, 29, 21, 71, 2, 12])
recipe_nu

,ingredient_id,ingredient_name,ingredient_amount,protein,fat,carbs,calorie
0,28,豚バラ肉,200g,28.8,70.8,0.2,732
1,1725,塩こしょう,小さじ1/4,0.2,0.1,1.0,5
2,76,卵,4個,24.4,20.4,0.8,284
3,1873,お湯,1000ml,0.0,0.0,0.0,0
4,342,薄力粉,大さじ1,0.7,0.1,6.8,31
5,1711,酒,大さじ1,0.1,0.0,0.7,10
6,206,砂糖,小さじ2,0.0,0.0,6.0,24
7,42,みりん,小さじ2,0.0,0.0,5.2,29
8,2242,しょうゆ,大さじ1.5,2.1,0.0,2.1,21
9,3517,サラダ油,小さじ2,0.0,0.0,8.0,71


In [23]:
recipe_nu=recipe.assign(protein =[42.2, 1.4, 0.2, 0.1,0.6, 0.4  ],
                                            fat =[0.5, 0.2, 0.1, 8.1, 0, 0.3 ],
                                            carbs =[0.2, 13.6, 1, 0,1, 5.8],
                                            calorie = [173, 62, 5, 70, 7, 20])
recipe_nu

,ingredient_id,ingredient_name,ingredient_amount,protein,fat,carbs,calorie
0,538,タラの切り身,2切れ,42.2,0.5,0.2,173
1,342,薄力粉,大さじ2,1.4,0.2,13.6,62
2,1725,塩こしょう,適量,0.2,0.1,1.0,5
3,1142,有塩バター,10g,0.1,8.1,0.0,70
4,189,パセリ,適量,0.6,0.0,1.0,7
5,34,レモン,1/4個,0.4,0.3,5.8,20


In [27]:
recipe_nu=recipe.assign(protein =[38.4, 0, 0.5, 0.2, 0.1, 1.4, 0, 0, 0],
                                            fat =[18.4, 0, 0.1, 0, 0, 0, 0, 0, 0],
                                            carbs =[0.4, 0, 4.5, 1.4, 7.8, 1.4, 6, 4, 0.1],
                                            calorie = [312, 0, 21, 20, 43, 14, 24, 35, 0])
recipe_nu

,ingredient_id,ingredient_name,ingredient_amount,protein,fat,carbs,calorie
0,1235,イワシ,2尾,38.4,18.4,0.4,312
1,178,塩,少々,0.0,0.0,0.0,0
2,342,薄力粉,小さじ2,0.5,0.1,4.5,21
3,1711,酒,大さじ2,0.2,0.0,1.4,20
4,42,みりん,大さじ1,0.1,0.0,7.8,43
5,2242,しょうゆ,大さじ1,1.4,0.0,1.4,14
6,619,きび砂糖,小さじ2,0.0,0.0,6.0,24
7,3517,サラダ油,小さじ1,0.0,0.0,4.0,35
8,99,大葉,1枚,0.0,0.0,0.1,0


In [40]:
print(round(recipe_nu['protein'].sum())),
print(round(recipe_nu['fat'].sum())),
print(round(recipe_nu['carbs'].sum())),
print(round(recipe_nu['calorie'].sum()))

47.0
14.0
22.0
379


In [35]:
recipe_nu=recipe.assign(protein =[76.8, 0, 0.6, 0, 0, 0.1 ],
                                            fat =[36.8, 0, 0.2, 0, 0, 0],
                                            carbs =[0.8,0, 8, 8, 0.2, 1.2],
                                            calorie = [624, 0, 25, 71, 0, 5])
recipe_nu

,ingredient_id,ingredient_name,ingredient_amount,protein,fat,carbs,calorie
0,1235,イワシ,4尾,76.8,36.8,0.8,624
1,178,塩,ふたつまみ,0.0,0.0,0.0,0
2,80,大根,100g,0.6,0.2,8.0,25
3,3517,サラダ油,大さじ1/2,0.0,0.0,8.0,71
4,99,大葉,2枚,0.0,0.0,0.2,0
5,34,レモン,2切れ,0.1,0.0,1.2,5


In [39]:
recipe_nu=recipe.assign(protein =[39.4, 2.6, 2.7, 0.2, 0.7, 0, 0.1, 1.4, 0.1],
                                            fat =[9, 0.3, 0.2, 0.1, 0.1, 4, 0, 0, 0],
                                            carbs =[0.2, 4.5, 7.6, 1, 6.8, 0, 0.7,1.4, 0.2],
                                            calorie = [224, 25, 34, 5, 31, 35, 10, 14, 1])
recipe_nu

,ingredient_id,ingredient_name,ingredient_amount,protein,fat,carbs,calorie
0,867,アジ,1尾,39.4,9.0,0.2,224
1,193,しめじ,1/2株,2.6,0.3,4.5,25
2,95,えのき,100g,2.7,0.2,7.6,34
3,1725,塩こしょう,適量,0.2,0.1,1.0,5
4,342,薄力粉,大さじ1,0.7,0.1,6.8,31
5,1142,有塩バター,5g,0.0,4.0,0.0,35
6,1711,酒,大さじ1,0.1,0.0,0.7,10
7,2242,しょうゆ,大さじ1,1.4,0.0,1.4,14
8,373,三つ葉,適量,0.1,0.0,0.2,1


In [13]:
sql = '''SELECT *
 FROM recipe
'''

In [16]:
connection = pymysql.connect(
     user='root', 
        password='na12199611', 
        host='localhost', 
        database='recipe' ,
    charset='utf8')

try:
    with connection.cursor() as cursor:
        cursor.execute(sql)
        rows = cursor.fetchall()

        new_result = [one for one in rows]
        train_data = pd.DataFrame(new_result)
        train_data.columns = ['recipe_id', 'recipe_name','category','preference', 'protein', 'fat', 'carbs', 'calorie']

finally:
    connection.close()

In [17]:
train_data

,recipe_id,recipe_name,category,preference,protein,fat,carbs,calorie
0,1,照り焼きチキン,1,5,27,29,18,457
1,2,肉巻き半熟卵,1,3,28,46,17,610
2,3,タラのムニエル,1,2,22,5,11,168
3,4,イワシの蒲焼き,1,2,41,18,26,469
4,5,イワシの塩焼き,1,4,39,18,9,362
...,...,...,...,...,...,...,...,...
85,86,土鍋で 鮎の炊き込みごはん,4,3,30,9,128,718
86,87,具沢山 五目チャーハン,4,4,37,33,82,813
87,88,あっという間に しらすとあさりの絶品焼き飯,4,3,9,9,29,251
88,89,土鍋で炊く 鮭とイクラの親子ごはん,4,3,15,3,67,392
